## Part 5: Updating Rows and Columns

Changing labels and values is how we clean and enrich data. This section shows patterns for renaming, mass-updating, safe assignment, and applying transformations.

---

### 1. Renaming columns in `df_small`

In [ ]:
import pandas as pd

people = {
    "first": ["Alice", "Bob", "Carol", "Sarah"],
    "last": ["Smith", "Jones", "Lee", "Jones"],
    "email": [
        "alice@example.com",
        "bob@example.com",
        "carol@example.com",
        "sarah@example.com",
    ],
    "uid": ["AS100293", "BJ240806", "CL150510", "SJ251203"],
    "year": [1993, 2006, 2010, 2003],
    "age": [32, 19, 15, 22],
}
df_small = pd.DataFrame(people)

In [ ]:
# 1a. Directly replace all column names
df_small.columns = ["first_name", "last_name", "email_address", "id_code", "byear", "age_2025"]
print("After assigning new column list:")
print(df_small.columns)

# 1b. Uppercase all column names via list comprehension
df_small.columns = [col.upper() for col in df_small.columns]
print("\nAfter uppercasing column names:")
print(df_small.columns)

# 1c. Modify names more flexibly: replace underscores with spaces (simulate if needed)
# First set back to some with underscores to demo
df_small.columns = [col.replace("_", " ") for col in df_small.columns]
print("\nAfter replacing underscores with spaces:")
print(df_small.columns)

# 1d. Rename only some columns (partial) using .rename
df_small = df_small.rename(columns={"FIRST NAME": "first", "LAST NAME": "last"})
print("\nAfter selective rename:")
print(df_small.columns)

---

### 2. Updating row values safely

In [ ]:
# Update entire row 1 (zero-based index) with new data using .loc
df_small.loc[1] = ["Robert", "Jones", "robert@example.com", "RJ010100", 2000, 25]
print("After replacing all of row 1:")
df_small

In [ ]:
# Update only some columns in a row
df_small.loc[2, ["email"]] = ["carol.modified@example.com"]
print("\nAfter updating only email for row 2:")
df_small

In [ ]:
# Update a single value using .at
df_small.at[0, "last"] = "Smythe"
print("\nAfter changing Alice's last name via .at:")
df_small

**Warning:**

Avoid chained assignment like this (can produce `SettingWithCopyWarning` and unreliable behavior):

In [ ]:
mask = df_small["last"] == "Jones"
# This is unsafe / may not persist:
df_small[mask]["last"] = "J."

Correct:

In [ ]:
df_small.loc[mask, "last"] = "J."

### 3. String methods on Series

In [ ]:
# Uppercase/lowercase transformations safely
df_small["email"] = df_small["email"].str.lower()  # vectorized lowercase
print("\nEmails after lowercasing:")
df_small["email"]

### 4. apply / applymap / map

In [ ]:
# apply on a Series: length of each email
email_lengths = df_small["email"].apply(len)
print("\nLength of each email (Series.apply):")
email_lengths

In [ ]:
# apply with a user-defined function
def shout(email):
    return email.upper()

In [ ]:
print("\nEmail shouted via apply:")
print(df_small["email"].apply(shout))

# with lambda
print("\nEmail first 5 chars using lambda:")
print(df_small["email"].apply(lambda x: x[:5]))

# Difference: df.apply(len) applies to each column (Series) -> gives number of non-null entries per column
print("\nUsing df.apply(len):")
print(df_small.apply(len))

# applymap applies to every element of the DataFrame
print("\nApplying str.lower to every string cell (applymap):")
print(df_small.applymap(lambda x: x.lower() if isinstance(x, str) else x))

### 5. map (elementwise with a mapping dict)

In [ ]:
# Suppose we want to abbreviate first names
df_small["first_abbrev"] = df_small["first"].map({"Alice": "A.", "Robert": "R.", "Carol": "C."})
print("\nAfter mapping first names to abbreviations:")
print(df_small[["first", "first_abbrev"]])

# Note: unmapped values become NaN; to avoid that, you can fill or use replace
df_small["first_abbrev_fallback"] = df_small["first"].replace({"Alice": "A.", "Robert": "R.", "Carol": "C."})
print("\nUsing replace (no NaNs for unmatched since exact replacements):")
print(df_small[["first", "first_abbrev_fallback"]])

### 6. Big DataFrame: rename column and use map

In [ ]:
df = pd.read_csv("data/survey_results_public.csv")

# Rename 'CompTotal' to 'Salary'
df = df.rename(columns={"CompTotal": "Salary"})
print("Columns after renaming compensation:")
print([c for c in df.columns if "Salary" in c])

# Example map usage: create a simplified label for some countries
country_abbrev = {
    "United States of America": "US",
    "Switzerland": "CH",
    "Germany": "DE",
    "India": "IN",
    "Canada": "CA",
}
df["country_code"] = df["Country"].map(country_abbrev)
print("\nSample country codes:")
print(df.loc[df["country_code"].notna(), ["Country", "country_code"]].head())

### 7. Combine first & last into a full name, then split back

In [ ]:
# Combine into full_name
df_small["full_name"] = df_small["first"] + " " + df_small["last"]
print("After combining:")
print(df_small)

# Drop the original first & last columns
df_small.drop(columns=["first", "last"], inplace=True)
print("\nAfter dropping first & last:")
print(df_small)

# Split full_name back into first & last
df_small[["first", "last"]] = df_small["full_name"].str.split(" ", expand=True)
print("\nAfter splitting full_name:")
print(df_small)



### 8. Appending rows & the ignore\_index=True quirk

In [ ]:
# Error: trying to append a dict without specifying the index
df_small.append({"first": "Tony"}, ignore_index=True)

In [ ]:
# Correct: using pd.concat
new_row = pd.DataFrame([{"first": "Tony"}])

# Method 1: Concatenate, resetting the index
df_small = pd.concat([df_small, new_row], ignore_index=True)
print(df_small)

# Method 2: Using .loc to append a new row
new_idx = len(df_small)  # next index
df_small.loc[new_idx] = {"first": "Tony"}

### 9. Appending another DataFrame

In [ ]:
# Create a smaller DataFrame with same columns
new_people = pd.DataFrame({
    "first": ["Diana", "Eli"],
    "last":  ["Prince", "Musk"],
    "email": ["diana@themiscira.com", "eli@space.com"],
    "full_name": ["Diana Prince", "Eli Musk"],
})

# Append new_people to df_small (resetting index for a clean result)
df_small = df_small.append(new_people, ignore_index=True)
print("\nCombined DataFrame:")
df_small

### 10. Removing rows by index and by condition

In [ ]:
# Remove a row by its index label, e.g., drop index 1
df_small = df_small.drop(index=1)
print("\nAfter dropping index=1:")
print(df_small)

# Remove rows where last name == "Lee"
mask_lee = df_small["last"] == "Lee"
df_no_lee = df_small.drop(index=df_small[mask_lee].index)
print("\nAfter dropping rows where last == 'Lee':")
df_no_lee

### Exercise for Part 5

#### Exercise 5.1
- Transform every column name so that it starts with an uppercase letter (e.g. `first` → `First`, `email` → `Email`).
- Replace each name and last name with the first letter followed by a dot using `apply()`.
- Extract the domain (after the `@`) from each email.

#### Exercise 5.2
- Create an `Adult` column to be `True` when the numeric age is **> 18**, and `False` otherwise (including NaN ages).